# Model Pruning Analysis

## EE4745 Neural Networks Final Project

This notebook demonstrates model pruning techniques for neural network compression and efficiency optimization.

### Objectives:
- Implement unstructured pruning techniques
- Analyze pruning effects on model performance
- Visualize pruning impact on network structure
- Evaluate robustness of pruned models
- Study the trade-offs between compression and accuracy

---

## 1. Setup and Configuration

Import libraries and set up the environment for model pruning analysis.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time
from collections import defaultdict
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.prune as prune
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

# Add src directory to path
sys.path.append('../src')

# Import custom modules
from dataset.sports_dataset import SportsDataset, get_dataloaders
from models.simple_cnn import create_simple_cnn
from models.resnet_small import create_resnet_small
from training.utils import set_seed, get_device, load_checkpoint, count_parameters
from pruning.unstructured import prune_model, evaluate_pruned_model
from attacks.fgsm import FGSM
from attacks.pgd import PGD

# Set style and configuration
plt.style.use('default')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

# Set seed for reproducibility
set_seed(42)

print("Model Pruning Analysis Setup")
print("=" * 35)
print(f"PyTorch version: {torch.__version__}")
device = get_device()
print(f"Device: {device}")
print(f"CUDA available: {torch.cuda.is_available()}")
print("Setup complete!")

## 2. Load Data and Pre-trained Models

Load dataset and pre-trained models for pruning analysis.

In [ ]:
# Dataset configuration
DATA_DIR = '../data'
IMAGE_SIZE = 32
BATCH_SIZE = 32
NUM_WORKERS = 2

# Load datasets
print("Loading datasets for pruning analysis...")
train_loader, val_loader, num_classes = get_dataloaders(
    data_dir=DATA_DIR,
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    num_workers=NUM_WORKERS
)

class_names = SportsDataset.CLASSES
print(f"Dataset loaded: {num_classes} classes")
print(f"Classes: {class_names}")
print(f"Validation samples: {len(val_loader.dataset)}")

# Create a smaller subset for faster pruning experiments
subset_size = 200
subset_indices = torch.randperm(len(val_loader.dataset))[:subset_size]
subset_dataset = torch.utils.data.Subset(val_loader.dataset, subset_indices)
subset_loader = DataLoader(subset_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Created subset with {subset_size} samples for detailed analysis")

### Load and Prepare Models for Pruning

In [ ]:
def load_or_create_model(model_type, checkpoint_path=None):
    """Load a pre-trained model or create a new one"""
    
    if model_type == 'SimpleCNN':
        model = create_simple_cnn(num_classes=num_classes, input_size=IMAGE_SIZE)
    elif model_type == 'ResNetSmall':
        model = create_resnet_small(num_classes=num_classes, input_size=IMAGE_SIZE)
    else:
        raise ValueError(f"Unknown model type: {model_type}")
    
    model.to(device)
    
    if checkpoint_path and os.path.exists(checkpoint_path):
        print(f"Loading checkpoint: {checkpoint_path}")
        try:
            load_checkpoint(checkpoint_path, model)
            print("✅ Checkpoint loaded successfully")
        except Exception as e:
            print(f"⚠️  Failed to load checkpoint: {e}")
            print("Using randomly initialized model")
    else:
        print(f"⚠️  No checkpoint found, using randomly initialized {model_type}")
    
    model.eval()
    return model

def evaluate_model_accuracy(model, dataloader):
    """Evaluate model accuracy on a dataloader"""
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    accuracy = 100.0 * correct / total
    return accuracy

# Define possible checkpoint paths
checkpoint_paths = {
    'SimpleCNN': '../checkpoints/SimpleCNN-best.pt',
    'ResNetSmall': '../checkpoints/ResNetSmall-best.pt'
}

# Load models
print("\nLoading models for pruning...")
models = {}

for model_type in ['SimpleCNN', 'ResNetSmall']:
    print(f"\nLoading {model_type}...")
    models[model_type] = load_or_create_model(model_type, checkpoint_paths.get(model_type))
    
    # Evaluate baseline accuracy
    baseline_accuracy = evaluate_model_accuracy(models[model_type], subset_loader)
    print(f"  Baseline accuracy: {baseline_accuracy:.2f}%")
    
    # Model size information
    total_params, trainable_params = count_parameters(models[model_type])
    print(f"  Total parameters: {total_params:,}")
    print(f"  Model size: {total_params * 4 / (1024**2):.2f} MB")

print(f"\nModels loaded: {list(models.keys())}")

## 3. Model Structure Analysis

Analyze the structure of models before pruning to understand pruning opportunities.

In [ ]:
def analyze_model_structure(model, model_name):
    """Analyze model structure and identify pruneable layers"""
    
    print(f"\nAnalyzing {model_name} structure...")
    print("=" * 40)
    
    layer_info = []
    total_params = 0
    pruneable_params = 0
    
    for name, module in model.named_modules():
        if isinstance(module, (nn.Linear, nn.Conv2d)):
            if hasattr(module, 'weight'):
                weight_params = module.weight.numel()
                bias_params = module.bias.numel() if module.bias is not None else 0
                layer_params = weight_params + bias_params
                
                layer_info.append({
                    'name': name,
                    'type': type(module).__name__,
                    'shape': list(module.weight.shape),
                    'weight_params': weight_params,
                    'bias_params': bias_params,
                    'total_params': layer_params,
                    'pruneable': True
                })
                
                total_params += layer_params
                pruneable_params += weight_params  # Only weights are typically pruned
                
                print(f"{name:30} | {type(module).__name__:10} | "
                      f"Shape: {str(list(module.weight.shape)):20} | "
                      f"Params: {layer_params:8,}")
    
    print(f"\nSummary:")
    print(f"  Total parameters in pruneable layers: {total_params:,}")
    print(f"  Pruneable weight parameters: {pruneable_params:,}")
    print(f"  Pruning potential: {pruneable_params/total_params*100:.1f}% of layer params")
    
    return layer_info

# Analyze structure of each model
model_structures = {}
for model_name, model in models.items():
    model_structures[model_name] = analyze_model_structure(model, model_name)

# Visualize model complexity
def visualize_model_complexity(model_structures):
    """Visualize parameter distribution across layers"""
    
    fig, axes = plt.subplots(1, len(model_structures), figsize=(6*len(model_structures), 6))
    
    if len(model_structures) == 1:
        axes = [axes]
    
    for idx, (model_name, structure) in enumerate(model_structures.items()):
        layer_names = [info['name'] for info in structure]
        param_counts = [info['total_params'] for info in structure]
        
        # Create pie chart of parameter distribution
        axes[idx].pie(param_counts, labels=layer_names, autopct='%1.1f%%', startangle=90)
        axes[idx].set_title(f'{model_name} - Parameter Distribution', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Create comparison table
    comparison_data = []
    for model_name, structure in model_structures.items():
        total_params = sum(info['total_params'] for info in structure)
        conv_params = sum(info['total_params'] for info in structure if info['type'] == 'Conv2d')
        linear_params = sum(info['total_params'] for info in structure if info['type'] == 'Linear')
        
        comparison_data.append({
            'Model': model_name,
            'Total Params': f"{total_params:,}",
            'Conv Params': f"{conv_params:,}",
            'Linear Params': f"{linear_params:,}",
            'Conv %': f"{conv_params/total_params*100:.1f}%",
            'Linear %': f"{linear_params/total_params*100:.1f}%"
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    print("\nModel Complexity Comparison:")
    print(comparison_df.to_string(index=False))

visualize_model_complexity(model_structures)

## 4. Unstructured Pruning Implementation

Implement magnitude-based unstructured pruning and analyze its effects.

In [ ]:
def apply_magnitude_pruning(model, pruning_ratio, layer_types=(nn.Conv2d, nn.Linear)):
    """
    Apply magnitude-based unstructured pruning to a model
    
    Args:
        model: PyTorch model to prune
        pruning_ratio: Fraction of weights to prune (0.0 to 1.0)
        layer_types: Types of layers to prune
    
    Returns:
        pruned_model: Model with pruning applied
        pruning_stats: Statistics about the pruning operation
    """
    
    # Create a copy of the model to avoid modifying the original
    pruned_model = copy.deepcopy(model)
    
    # Collect all layers to prune
    modules_to_prune = []
    
    for name, module in pruned_model.named_modules():
        if isinstance(module, layer_types):
            modules_to_prune.append((module, 'weight'))
    
    # Apply global magnitude pruning
    prune.global_unstructured(
        modules_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=pruning_ratio
    )
    
    # Calculate pruning statistics
    total_params = 0
    pruned_params = 0
    layer_stats = []
    
    for name, module in pruned_model.named_modules():
        if isinstance(module, layer_types) and hasattr(module, 'weight_mask'):
            mask = module.weight_mask
            total_weights = mask.numel()
            remaining_weights = mask.sum().item()
            pruned_weights = total_weights - remaining_weights
            
            layer_stats.append({
                'name': name,
                'type': type(module).__name__,
                'total_weights': total_weights,
                'pruned_weights': pruned_weights,
                'remaining_weights': remaining_weights,
                'pruning_ratio': pruned_weights / total_weights,
                'sparsity': pruned_weights / total_weights
            })
            
            total_params += total_weights
            pruned_params += pruned_weights
    
    overall_sparsity = pruned_params / total_params if total_params > 0 else 0
    
    pruning_stats = {
        'target_ratio': pruning_ratio,
        'actual_sparsity': overall_sparsity,
        'total_params': total_params,
        'pruned_params': pruned_params,
        'remaining_params': total_params - pruned_params,
        'layer_stats': layer_stats
    }
    
    return pruned_model, pruning_stats

def make_pruning_permanent(model):
    """Make pruning permanent by removing the masks and zeroed weights"""
    for module in model.modules():
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            if hasattr(module, 'weight_mask'):
                prune.remove(module, 'weight')
    return model

def evaluate_pruning_impact(original_model, pruned_model, dataloader, model_name, pruning_ratio):
    """Evaluate the impact of pruning on model performance"""
    
    print(f"\nEvaluating pruning impact for {model_name} (ratio: {pruning_ratio:.1%})")
    
    # Evaluate both models
    original_acc = evaluate_model_accuracy(original_model, dataloader)
    pruned_acc = evaluate_model_accuracy(pruned_model, dataloader)
    
    # Calculate model sizes
    original_params = sum(p.numel() for p in original_model.parameters())
    
    # Count actual non-zero parameters in pruned model
    pruned_nonzero = 0
    for module in pruned_model.modules():
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            if hasattr(module, 'weight_mask'):
                pruned_nonzero += module.weight_mask.sum().item()
                if hasattr(module, 'bias') and module.bias is not None:
                    pruned_nonzero += module.bias.numel()
            else:
                pruned_nonzero += sum(p.numel() for p in module.parameters())
    
    compression_ratio = original_params / pruned_nonzero if pruned_nonzero > 0 else float('inf')
    accuracy_drop = original_acc - pruned_acc
    
    results = {
        'model_name': model_name,
        'pruning_ratio': pruning_ratio,
        'original_accuracy': original_acc,
        'pruned_accuracy': pruned_acc,
        'accuracy_drop': accuracy_drop,
        'original_params': original_params,
        'pruned_nonzero_params': int(pruned_nonzero),
        'compression_ratio': compression_ratio,
        'size_reduction': (1 - pruned_nonzero/original_params) * 100
    }
    
    print(f"  Original accuracy: {original_acc:.2f}%")
    print(f"  Pruned accuracy: {pruned_acc:.2f}%")
    print(f"  Accuracy drop: {accuracy_drop:.2f}%")
    print(f"  Compression ratio: {compression_ratio:.2f}x")
    print(f"  Size reduction: {results['size_reduction']:.1f}%")
    
    return results

# Test different pruning ratios
pruning_ratios = [0.1, 0.3, 0.5, 0.7, 0.9]
pruning_results = []

print("MAGNITUDE-BASED PRUNING ANALYSIS")
print("=" * 40)

for model_name, original_model in models.items():
    print(f"\nPruning {model_name}...")
    
    for ratio in pruning_ratios:
        # Apply pruning
        pruned_model, pruning_stats = apply_magnitude_pruning(original_model, ratio)
        
        # Evaluate impact
        result = evaluate_pruning_impact(
            original_model, pruned_model, subset_loader, model_name, ratio
        )
        
        # Add pruning statistics
        result.update({
            'actual_sparsity': pruning_stats['actual_sparsity'],
            'target_sparsity': pruning_stats['target_ratio']
        })
        
        pruning_results.append(result)

# Convert to DataFrame for analysis
pruning_df = pd.DataFrame(pruning_results)
print("\nPruning analysis complete!")

## 5. Pruning Results Visualization

Visualize the effects of pruning on model performance and compression.

In [ ]:
def visualize_pruning_results(pruning_df):
    """Visualize pruning results across different ratios and models"""
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Model Pruning Analysis', fontsize=16, fontweight='bold')
    
    models = pruning_df['model_name'].unique()
    colors = ['blue', 'red', 'green', 'orange']
    
    # Plot 1: Accuracy vs Pruning Ratio
    for i, model in enumerate(models):
        model_data = pruning_df[pruning_df['model_name'] == model]
        
        # Original accuracy line
        orig_acc = model_data['original_accuracy'].iloc[0]
        axes[0, 0].axhline(y=orig_acc, color=colors[i], linestyle='--', alpha=0.5, 
                          label=f'{model} Original')
        
        # Pruned accuracy
        axes[0, 0].plot(model_data['pruning_ratio'], model_data['pruned_accuracy'], 
                       'o-', color=colors[i], label=f'{model} Pruned', linewidth=2, markersize=6)
    
    axes[0, 0].set_title('Accuracy vs Pruning Ratio', fontweight='bold')
    axes[0, 0].set_xlabel('Pruning Ratio')
    axes[0, 0].set_ylabel('Accuracy (%)')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: Compression Ratio vs Accuracy Drop
    for i, model in enumerate(models):
        model_data = pruning_df[pruning_df['model_name'] == model]
        axes[0, 1].scatter(model_data['compression_ratio'], model_data['accuracy_drop'], 
                          s=100, color=colors[i], label=model, alpha=0.7)
        
        # Add pruning ratio annotations
        for _, row in model_data.iterrows():
            axes[0, 1].annotate(f'{row["pruning_ratio"]:.1f}', 
                               (row['compression_ratio'], row['accuracy_drop']),
                               xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    axes[0, 1].set_title('Compression vs Accuracy Trade-off', fontweight='bold')
    axes[0, 1].set_xlabel('Compression Ratio')
    axes[0, 1].set_ylabel('Accuracy Drop (%)')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Size Reduction vs Pruning Ratio
    for i, model in enumerate(models):
        model_data = pruning_df[pruning_df['model_name'] == model]
        axes[1, 0].plot(model_data['pruning_ratio'], model_data['size_reduction'], 
                       's-', color=colors[i], label=model, linewidth=2, markersize=6)
    
    axes[1, 0].set_title('Model Size Reduction', fontweight='bold')
    axes[1, 0].set_xlabel('Pruning Ratio')
    axes[1, 0].set_ylabel('Size Reduction (%)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 4: Efficiency Score (Accuracy / Model Size)
    for i, model in enumerate(models):
        model_data = pruning_df[pruning_df['model_name'] == model]
        
        # Calculate efficiency score
        efficiency_score = (model_data['pruned_accuracy'] / model_data['pruned_nonzero_params']) * 1000000
        
        axes[1, 1].plot(model_data['pruning_ratio'], efficiency_score, 
                       '^-', color=colors[i], label=model, linewidth=2, markersize=6)
    
    axes[1, 1].set_title('Model Efficiency Score\n(Accuracy per Million Parameters)', fontweight='bold')
    axes[1, 1].set_xlabel('Pruning Ratio')
    axes[1, 1].set_ylabel('Efficiency Score')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def create_pruning_summary_table(pruning_df):
    """Create a summary table of pruning results"""
    
    summary_data = []
    
    for model in pruning_df['model_name'].unique():
        model_data = pruning_df[pruning_df['model_name'] == model]
        
        # Find best pruning ratio (minimal accuracy drop with good compression)
        # Use a composite score: compression_ratio / (1 + accuracy_drop)
        model_data = model_data.copy()
        model_data['composite_score'] = (model_data['compression_ratio'] / 
                                       (1 + model_data['accuracy_drop']))
        
        best_idx = model_data['composite_score'].idxmax()
        best_result = model_data.loc[best_idx]
        
        summary_data.append({
            'Model': model,
            'Original Acc (%)': f"{best_result['original_accuracy']:.2f}",
            'Best Pruning Ratio': f"{best_result['pruning_ratio']:.1f}",
            'Pruned Acc (%)': f"{best_result['pruned_accuracy']:.2f}",
            'Accuracy Drop (%)': f"{best_result['accuracy_drop']:.2f}",
            'Compression': f"{best_result['compression_ratio']:.1f}x",
            'Size Reduction (%)': f"{best_result['size_reduction']:.1f}",
            'Efficiency Score': f"{(best_result['pruned_accuracy'] / best_result['pruned_nonzero_params']) * 1000000:.2f}"
        })
    
    summary_df = pd.DataFrame(summary_data)
    
    print("\nPruning Summary Table (Best Trade-off):")
    print("=" * 60)
    print(summary_df.to_string(index=False))
    
    return summary_df

# Visualize pruning results
visualize_pruning_results(pruning_df)

# Create summary table
summary_table = create_pruning_summary_table(pruning_df)

# Print detailed results for each model
print("\n\nDetailed Pruning Results:")
print("=" * 50)

for model in pruning_df['model_name'].unique():
    model_data = pruning_df[pruning_df['model_name'] == model]
    print(f"\n{model}:")
    print(f"{'Ratio':>8} {'Accuracy':>10} {'Drop':>8} {'Compression':>12} {'Size Red.':>10}")
    print("-" * 55)
    
    for _, row in model_data.iterrows():
        print(f"{row['pruning_ratio']:>8.1f} {row['pruned_accuracy']:>9.2f}% "
              f"{row['accuracy_drop']:>7.2f}% {row['compression_ratio']:>10.1f}x "
              f"{row['size_reduction']:>9.1f}%")

## 6. Layer-wise Pruning Analysis

Analyze how pruning affects different layers of the network.

In [ ]:
def analyze_layerwise_pruning(model, pruning_ratio=0.5):
    """Analyze how pruning affects different layers"""
    
    # Apply pruning to get detailed statistics
    pruned_model, pruning_stats = apply_magnitude_pruning(model, pruning_ratio)
    
    layer_stats = pruning_stats['layer_stats']
    
    print(f"\nLayer-wise Pruning Analysis (ratio: {pruning_ratio:.1%})")
    print("=" * 60)
    print(f"{'Layer':30} {'Type':10} {'Total':>10} {'Pruned':>10} {'Sparsity':>10}")
    print("-" * 75)
    
    for stat in layer_stats:
        print(f"{stat['name']:30} {stat['type']:10} {stat['total_weights']:>10,} "
              f"{stat['pruned_weights']:>10,} {stat['sparsity']:>9.1%}")
    
    return layer_stats, pruned_model

def visualize_layerwise_sparsity(layer_stats_dict):
    """Visualize sparsity across different layers"""
    
    fig, axes = plt.subplots(1, len(layer_stats_dict), figsize=(6*len(layer_stats_dict), 6))
    
    if len(layer_stats_dict) == 1:
        axes = [axes]
    
    for idx, (model_name, layer_stats) in enumerate(layer_stats_dict.items()):
        layer_names = [stat['name'].split('.')[-1] for stat in layer_stats]  # Short names
        sparsities = [stat['sparsity'] * 100 for stat in layer_stats]
        layer_types = [stat['type'] for stat in layer_stats]
        
        # Color by layer type
        colors = ['skyblue' if t == 'Conv2d' else 'lightcoral' for t in layer_types]
        
        bars = axes[idx].bar(range(len(layer_names)), sparsities, color=colors, alpha=0.7)
        axes[idx].set_title(f'{model_name} - Layer-wise Sparsity', fontweight='bold')
        axes[idx].set_xlabel('Layers')
        axes[idx].set_ylabel('Sparsity (%)')
        axes[idx].set_xticks(range(len(layer_names)))
        axes[idx].set_xticklabels(layer_names, rotation=45, ha='right')
        axes[idx].grid(True, alpha=0.3)
        
        # Add value labels on bars
        for bar, sparsity in zip(bars, sparsities):
            height = bar.get_height()
            axes[idx].text(bar.get_x() + bar.get_width()/2., height + 1,
                          f'{sparsity:.1f}%', ha='center', va='bottom', fontsize=8)
        
        # Create legend
        conv_patch = plt.Rectangle((0,0), 1, 1, facecolor='skyblue', alpha=0.7, label='Conv2d')
        linear_patch = plt.Rectangle((0,0), 1, 1, facecolor='lightcoral', alpha=0.7, label='Linear')
        axes[idx].legend(handles=[conv_patch, linear_patch])
    
    plt.tight_layout()
    plt.show()

# Analyze layer-wise pruning for each model
layer_stats_dict = {}
pruned_models_50 = {}

for model_name, model in models.items():
    print(f"\nAnalyzing layer-wise pruning for {model_name}...")
    layer_stats, pruned_model = analyze_layerwise_pruning(model, pruning_ratio=0.5)
    layer_stats_dict[model_name] = layer_stats
    pruned_models_50[model_name] = pruned_model

# Visualize layer-wise sparsity
visualize_layerwise_sparsity(layer_stats_dict)

# Analyze parameter distribution after pruning
print("\nParameter Distribution Analysis:")
print("=" * 40)

for model_name, layer_stats in layer_stats_dict.items():
    total_original = sum(stat['total_weights'] for stat in layer_stats)
    total_remaining = sum(stat['remaining_weights'] for stat in layer_stats)
    
    conv_original = sum(stat['total_weights'] for stat in layer_stats if stat['type'] == 'Conv2d')
    conv_remaining = sum(stat['remaining_weights'] for stat in layer_stats if stat['type'] == 'Conv2d')
    
    linear_original = sum(stat['total_weights'] for stat in layer_stats if stat['type'] == 'Linear')
    linear_remaining = sum(stat['remaining_weights'] for stat in layer_stats if stat['type'] == 'Linear')
    
    print(f"\n{model_name}:")
    print(f"  Total parameters: {total_original:,} → {total_remaining:,} ({total_remaining/total_original:.1%})")
    print(f"  Conv parameters: {conv_original:,} → {conv_remaining:,} ({conv_remaining/conv_original:.1%})")
    print(f"  Linear parameters: {linear_original:,} → {linear_remaining:,} ({linear_remaining/linear_original:.1%})")

print("\nLayer-wise analysis complete!")

## 7. Robustness Analysis of Pruned Models

Analyze how pruning affects model robustness to adversarial attacks.

In [ ]:
def evaluate_adversarial_robustness(model, dataloader, model_name, attack_configs):
    """Evaluate model robustness against adversarial attacks"""
    
    results = {}
    
    # Clean accuracy
    clean_acc = evaluate_model_accuracy(model, dataloader)
    results['clean'] = clean_acc
    
    print(f"\nEvaluating adversarial robustness for {model_name}...")
    print(f"Clean accuracy: {clean_acc:.2f}%")
    
    for attack_name, config in attack_configs.items():
        if config['type'] == 'fgsm':
            attack = FGSM(model, device=device)
        elif config['type'] == 'pgd':
            attack = PGD(model, device=device, steps=config.get('steps', 10), 
                        step_size=config.get('step_size', 0.01))
        else:
            continue
        
        # Evaluate attack
        correct = 0
        total = 0
        
        model.eval()
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            
            # Generate adversarial examples
            adv_images = attack.attack(images, labels, epsilon=config['epsilon'])
            
            # Test on adversarial examples
            with torch.no_grad():
                outputs = model(adv_images)
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
        
        adv_acc = 100.0 * correct / total
        results[attack_name] = adv_acc
        
        print(f"{attack_name} accuracy (ε={config['epsilon']}): {adv_acc:.2f}%")
    
    return results

def compare_robustness_before_after_pruning(original_models, pruned_models, dataloader):
    """Compare robustness before and after pruning"""
    
    # Attack configurations to test
    attack_configs = {
        'FGSM_mild': {'type': 'fgsm', 'epsilon': 0.05},
        'FGSM_strong': {'type': 'fgsm', 'epsilon': 0.1},
        'PGD_mild': {'type': 'pgd', 'epsilon': 0.05, 'steps': 10, 'step_size': 0.005},
        'PGD_strong': {'type': 'pgd', 'epsilon': 0.1, 'steps': 10, 'step_size': 0.01}
    }
    
    robustness_comparison = []
    
    for model_name in original_models.keys():
        if model_name in pruned_models:
            print(f"\n{'='*50}")
            print(f"ROBUSTNESS COMPARISON: {model_name}")
            print(f"{'='*50}")
            
            # Evaluate original model
            original_results = evaluate_adversarial_robustness(
                original_models[model_name], dataloader, f"{model_name}_original", attack_configs
            )
            
            # Evaluate pruned model
            pruned_results = evaluate_adversarial_robustness(
                pruned_models[model_name], dataloader, f"{model_name}_pruned", attack_configs
            )
            
            # Calculate changes
            for attack_name in attack_configs.keys():
                original_acc = original_results.get(attack_name, 0)
                pruned_acc = pruned_results.get(attack_name, 0)
                change = pruned_acc - original_acc
                
                robustness_comparison.append({
                    'model': model_name,
                    'attack': attack_name,
                    'original_acc': original_acc,
                    'pruned_acc': pruned_acc,
                    'change': change
                })
    
    return robustness_comparison

def visualize_robustness_comparison(robustness_data):
    """Visualize robustness comparison before and after pruning"""
    
    df = pd.DataFrame(robustness_data)
    
    if df.empty:
        print("No robustness data to visualize.")
        return
    
    models = df['model'].unique()
    attacks = df['attack'].unique()
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Accuracy comparison
    x = np.arange(len(attacks))
    width = 0.35
    
    for i, model in enumerate(models):
        model_data = df[df['model'] == model]
        
        if not model_data.empty:
            original_accs = [model_data[model_data['attack'] == attack]['original_acc'].iloc[0] 
                           for attack in attacks if not model_data[model_data['attack'] == attack].empty]
            pruned_accs = [model_data[model_data['attack'] == attack]['pruned_acc'].iloc[0] 
                         for attack in attacks if not model_data[model_data['attack'] == attack].empty]
            
            if len(original_accs) == len(attacks):
                offset = (i - len(models)/2 + 0.5) * width / len(models)
                axes[0].bar(x + offset - width/4, original_accs, width/2, 
                           label=f'{model} Original', alpha=0.8)
                axes[0].bar(x + offset + width/4, pruned_accs, width/2, 
                           label=f'{model} Pruned', alpha=0.8)
    
    axes[0].set_title('Adversarial Robustness: Original vs Pruned', fontweight='bold')
    axes[0].set_xlabel('Attack Type')
    axes[0].set_ylabel('Accuracy (%)')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(attacks, rotation=45, ha='right')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot 2: Change in robustness
    for i, model in enumerate(models):
        model_data = df[df['model'] == model]
        
        if not model_data.empty:
            changes = [model_data[model_data['attack'] == attack]['change'].iloc[0] 
                      for attack in attacks if not model_data[model_data['attack'] == attack].empty]
            
            if len(changes) == len(attacks):
                offset = (i - len(models)/2 + 0.5) * width / len(models)
                bars = axes[1].bar(x + offset, changes, width/len(models), 
                                  label=model, alpha=0.8)
                
                # Color bars based on positive/negative change
                for bar, change in zip(bars, changes):
                    if change < 0:
                        bar.set_color('red')
                    else:
                        bar.set_color('green')
    
    axes[1].axhline(y=0, color='black', linestyle='-', alpha=0.5)
    axes[1].set_title('Change in Robustness After Pruning', fontweight='bold')
    axes[1].set_xlabel('Attack Type')
    axes[1].set_ylabel('Accuracy Change (%)')
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(attacks, rotation=45, ha='right')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Compare robustness before and after pruning
print("ROBUSTNESS ANALYSIS OF PRUNED MODELS")
print("=" * 45)

# Use the 50% pruned models from the previous section
if pruned_models_50:
    robustness_comparison = compare_robustness_before_after_pruning(
        models, pruned_models_50, subset_loader
    )
    
    # Visualize robustness comparison
    visualize_robustness_comparison(robustness_comparison)
    
    # Print summary
    print("\nRobustness Analysis Summary:")
    print("=" * 35)
    
    for model_name in models.keys():
        if model_name in pruned_models_50:
            model_data = [r for r in robustness_comparison if r['model'] == model_name]
            
            if model_data:
                avg_change = np.mean([r['change'] for r in model_data])
                print(f"\n{model_name}:")
                print(f"  Average robustness change: {avg_change:+.2f}%")
                
                if avg_change > 1:
                    print(f"  ✅ Improved robustness after pruning")
                elif avg_change > -2:
                    print(f"  ⚠️  Minimal impact on robustness")
                else:
                    print(f"  ❌ Decreased robustness after pruning")
                
                for r in model_data:
                    print(f"    {r['attack']}: {r['change']:+.2f}%")

else:
    print("No pruned models available for robustness analysis.")

print("\nRobustness analysis complete!")

## 8. Advanced Pruning Techniques

Explore structured pruning and gradual pruning techniques.

In [ ]:
def structured_pruning(model, pruning_ratio):
    """Apply structured pruning by removing entire channels"""
    
    pruned_model = copy.deepcopy(model)
    
    # For demonstration, we'll apply channel-wise pruning to convolutional layers
    conv_modules = [(name, module) for name, module in pruned_model.named_modules() 
                   if isinstance(module, nn.Conv2d)]
    
    pruning_stats = []
    
    for name, module in conv_modules:
        if module.weight.size(0) > 1:  # Don't prune if only 1 channel
            # Calculate channel importance (L1 norm)
            channel_importance = torch.sum(torch.abs(module.weight.view(module.weight.size(0), -1)), dim=1)
            
            # Determine number of channels to prune
            num_channels = module.weight.size(0)
            num_to_prune = int(num_channels * pruning_ratio)
            
            if num_to_prune > 0 and num_to_prune < num_channels:
                # Get indices of least important channels
                _, indices_to_prune = torch.topk(channel_importance, num_to_prune, largest=False)
                
                # Create mask for remaining channels
                channel_mask = torch.ones(num_channels, dtype=torch.bool)
                channel_mask[indices_to_prune] = False
                
                pruning_stats.append({
                    'layer': name,
                    'original_channels': num_channels,
                    'pruned_channels': num_to_prune,
                    'remaining_channels': num_channels - num_to_prune
                })
    
    return pruned_model, pruning_stats

def gradual_pruning_simulation(model, dataloader, target_ratio=0.8, steps=5):
    """Simulate gradual pruning process"""
    
    current_model = copy.deepcopy(model)
    gradual_results = []
    
    # Calculate pruning ratio per step
    step_ratio = target_ratio / steps
    
    print(f"\nGradual Pruning Simulation (target: {target_ratio:.1%}, steps: {steps})")
    print("=" * 60)
    
    # Initial evaluation
    initial_acc = evaluate_model_accuracy(current_model, dataloader)
    initial_params = sum(p.numel() for p in current_model.parameters())
    
    gradual_results.append({
        'step': 0,
        'pruning_ratio': 0.0,
        'cumulative_ratio': 0.0,
        'accuracy': initial_acc,
        'remaining_params': initial_params,
        'compression_ratio': 1.0
    })
    
    print(f"Step 0 (Initial): Acc={initial_acc:.2f}%, Params={initial_params:,}")
    
    cumulative_pruned = 0.0
    
    for step in range(1, steps + 1):
        # Apply pruning for this step
        current_model, _ = apply_magnitude_pruning(current_model, step_ratio)
        
        # Make pruning permanent to get actual parameter reduction
        temp_model = copy.deepcopy(current_model)
        make_pruning_permanent(temp_model)
        
        # Evaluate
        step_acc = evaluate_model_accuracy(current_model, dataloader)
        
        # Count remaining parameters
        remaining_params = 0
        for module in current_model.modules():
            if isinstance(module, (nn.Conv2d, nn.Linear)):
                if hasattr(module, 'weight_mask'):
                    remaining_params += module.weight_mask.sum().item()
                    if hasattr(module, 'bias') and module.bias is not None:
                        remaining_params += module.bias.numel()
                else:
                    remaining_params += sum(p.numel() for p in module.parameters())
        
        cumulative_pruned = 1 - (remaining_params / initial_params)
        compression_ratio = initial_params / remaining_params
        
        gradual_results.append({
            'step': step,
            'pruning_ratio': step_ratio,
            'cumulative_ratio': cumulative_pruned,
            'accuracy': step_acc,
            'remaining_params': int(remaining_params),
            'compression_ratio': compression_ratio
        })
        
        print(f"Step {step}: Acc={step_acc:.2f}%, "
              f"Cumulative pruning={cumulative_pruned:.1%}, "
              f"Compression={compression_ratio:.1f}x")
    
    return gradual_results

def visualize_advanced_pruning(gradual_results, structured_results=None):
    """Visualize results from advanced pruning techniques"""
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Gradual pruning progression
    steps = [r['step'] for r in gradual_results]
    accuracies = [r['accuracy'] for r in gradual_results]
    cumulative_ratios = [r['cumulative_ratio'] * 100 for r in gradual_results]
    
    ax1 = axes[0]
    color1 = 'tab:blue'
    ax1.set_xlabel('Pruning Step')
    ax1.set_ylabel('Accuracy (%)', color=color1)
    line1 = ax1.plot(steps, accuracies, 'o-', color=color1, label='Accuracy', linewidth=2, markersize=6)
    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.grid(True, alpha=0.3)
    
    ax2 = ax1.twinx()
    color2 = 'tab:red'
    ax2.set_ylabel('Cumulative Pruning (%)', color=color2)
    line2 = ax2.plot(steps, cumulative_ratios, 's-', color=color2, label='Cumulative Pruning', 
                    linewidth=2, markersize=6)
    ax2.tick_params(axis='y', labelcolor=color2)
    
    # Add legends
    lines = line1 + line2
    labels = [l.get_label() for l in lines]
    ax1.legend(lines, labels, loc='center right')
    
    axes[0].set_title('Gradual Pruning Progression', fontweight='bold')
    
    # Plot 2: Comparison of different pruning ratios
    if len(gradual_results) > 1:
        compression_ratios = [r['compression_ratio'] for r in gradual_results]
        
        # Create scatter plot
        scatter = axes[1].scatter(compression_ratios, accuracies, 
                                c=steps, cmap='viridis', s=100, alpha=0.7)
        
        # Add step labels
        for i, (comp, acc, step) in enumerate(zip(compression_ratios, accuracies, steps)):
            axes[1].annotate(f'Step {step}', (comp, acc), 
                           xytext=(5, 5), textcoords='offset points', fontsize=8)
        
        axes[1].set_xlabel('Compression Ratio')
        axes[1].set_ylabel('Accuracy (%)')
        axes[1].set_title('Accuracy vs Compression Trade-off', fontweight='bold')
        axes[1].grid(True, alpha=0.3)
        
        # Add colorbar
        cbar = plt.colorbar(scatter, ax=axes[1])
        cbar.set_label('Pruning Step')
    
    plt.tight_layout()
    plt.show()

# Test advanced pruning techniques
print("ADVANCED PRUNING TECHNIQUES")
print("=" * 35)

# Select a model for advanced pruning
if models:
    test_model_name = list(models.keys())[0]  # Use first available model
    test_model = models[test_model_name]
    
    print(f"Testing advanced pruning on {test_model_name}...")
    
    # 1. Structured pruning (conceptual - actual implementation would be more complex)
    print(f"\n1. Structured Pruning Analysis:")
    structured_model, structured_stats = structured_pruning(test_model, 0.3)
    
    if structured_stats:
        print("\nStructured Pruning Results:")
        for stat in structured_stats:
            print(f"  {stat['layer']}: {stat['original_channels']} → "
                  f"{stat['remaining_channels']} channels "
                  f"({stat['pruned_channels']} pruned)")
    
    # 2. Gradual pruning simulation
    print(f"\n2. Gradual Pruning Simulation:")
    gradual_results = gradual_pruning_simulation(test_model, subset_loader, 
                                                target_ratio=0.7, steps=4)
    
    # Visualize results
    visualize_advanced_pruning(gradual_results)
    
    # Analysis of gradual vs one-shot pruning
    print("\nGradual vs One-Shot Pruning Comparison:")
    print("-" * 45)
    
    final_gradual = gradual_results[-1]
    
    # Apply one-shot pruning with same target ratio
    target_ratio = final_gradual['cumulative_ratio']
    oneshot_model, _ = apply_magnitude_pruning(test_model, target_ratio)
    oneshot_acc = evaluate_model_accuracy(oneshot_model, subset_loader)
    
    print(f"Target pruning ratio: {target_ratio:.1%}")
    print(f"Gradual pruning final accuracy: {final_gradual['accuracy']:.2f}%")
    print(f"One-shot pruning accuracy: {oneshot_acc:.2f}%")
    print(f"Gradual advantage: {final_gradual['accuracy'] - oneshot_acc:+.2f}%")
    
    if final_gradual['accuracy'] > oneshot_acc:
        print("✅ Gradual pruning shows better accuracy retention")
    else:
        print("⚠️  One-shot pruning performed similarly or better")

else:
    print("No models available for advanced pruning analysis.")

print("\nAdvanced pruning analysis complete!")

## 9. Inference Speed and Memory Analysis

Analyze the practical benefits of pruning in terms of inference speed and memory usage.

In [ ]:
def measure_inference_speed(model, input_size=(1, 3, 32, 32), num_trials=100, warmup_trials=10):
    """Measure model inference speed"""
    
    model.eval()
    dummy_input = torch.randn(*input_size).to(device)
    
    # Warmup
    with torch.no_grad():
        for _ in range(warmup_trials):
            _ = model(dummy_input)
    
    # Actual timing
    torch.cuda.synchronize() if torch.cuda.is_available() else None
    start_time = time.time()
    
    with torch.no_grad():
        for _ in range(num_trials):
            _ = model(dummy_input)
    
    torch.cuda.synchronize() if torch.cuda.is_available() else None
    end_time = time.time()
    
    avg_time_ms = (end_time - start_time) * 1000 / num_trials
    return avg_time_ms

def measure_memory_usage(model, input_size=(1, 3, 32, 32)):
    """Measure model memory usage"""
    
    # Model parameter memory
    param_memory_mb = sum(p.numel() for p in model.parameters()) * 4 / (1024**2)  # 4 bytes per float32
    
    # Activation memory (approximate)
    dummy_input = torch.randn(*input_size).to(device)
    
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()
        
        model.eval()
        with torch.no_grad():
            _ = model(dummy_input)
        
        peak_memory_mb = torch.cuda.max_memory_allocated() / (1024**2)
        activation_memory_mb = peak_memory_mb - param_memory_mb
    else:
        # Rough estimation for CPU
        activation_memory_mb = param_memory_mb * 0.5  # Very rough estimate
        peak_memory_mb = param_memory_mb + activation_memory_mb
    
    return {
        'param_memory_mb': param_memory_mb,
        'activation_memory_mb': max(0, activation_memory_mb),
        'total_memory_mb': peak_memory_mb
    }

def comprehensive_performance_analysis(original_models, pruned_models_dict):
    """Comprehensive analysis of performance improvements from pruning"""
    
    performance_data = []
    
    print("\nCOMPREHENSIVE PERFORMANCE ANALYSIS")
    print("=" * 45)
    
    for model_name, original_model in original_models.items():
        print(f"\nAnalyzing {model_name}...")
        
        # Original model analysis
        original_speed = measure_inference_speed(original_model)
        original_memory = measure_memory_usage(original_model)
        original_params = sum(p.numel() for p in original_model.parameters())
        original_acc = evaluate_model_accuracy(original_model, subset_loader)
        
        performance_data.append({
            'model': model_name,
            'variant': 'Original',
            'pruning_ratio': 0.0,
            'accuracy': original_acc,
            'inference_time_ms': original_speed,
            'param_memory_mb': original_memory['param_memory_mb'],
            'total_memory_mb': original_memory['total_memory_mb'],
            'parameters': original_params,
            'speedup': 1.0,
            'memory_reduction': 0.0
        })
        
        print(f"  Original - Acc: {original_acc:.2f}%, "
              f"Speed: {original_speed:.2f}ms, "
              f"Memory: {original_memory['total_memory_mb']:.1f}MB")
        
        # Analyze pruned variants if available
        if model_name in pruned_models_dict:
            pruned_model = pruned_models_dict[model_name]
            
            pruned_speed = measure_inference_speed(pruned_model)
            pruned_memory = measure_memory_usage(pruned_model)
            pruned_acc = evaluate_model_accuracy(pruned_model, subset_loader)
            
            # Count effective parameters (non-masked)
            pruned_params = 0
            for module in pruned_model.modules():
                if isinstance(module, (nn.Conv2d, nn.Linear)):
                    if hasattr(module, 'weight_mask'):
                        pruned_params += module.weight_mask.sum().item()
                        if hasattr(module, 'bias') and module.bias is not None:
                            pruned_params += module.bias.numel()
                    else:
                        pruned_params += sum(p.numel() for p in module.parameters())
            
            speedup = original_speed / pruned_speed if pruned_speed > 0 else 1.0
            memory_reduction = ((original_memory['total_memory_mb'] - pruned_memory['total_memory_mb']) / 
                              original_memory['total_memory_mb']) * 100
            
            performance_data.append({
                'model': model_name,
                'variant': 'Pruned (50%)',
                'pruning_ratio': 0.5,
                'accuracy': pruned_acc,
                'inference_time_ms': pruned_speed,
                'param_memory_mb': pruned_memory['param_memory_mb'],
                'total_memory_mb': pruned_memory['total_memory_mb'],
                'parameters': int(pruned_params),
                'speedup': speedup,
                'memory_reduction': memory_reduction
            })
            
            print(f"  Pruned   - Acc: {pruned_acc:.2f}%, "
                  f"Speed: {pruned_speed:.2f}ms ({speedup:.1f}x speedup), "
                  f"Memory: {pruned_memory['total_memory_mb']:.1f}MB ({memory_reduction:.1f}% reduction)")
    
    return performance_data

def visualize_performance_analysis(performance_data):
    """Visualize performance analysis results"""
    
    df = pd.DataFrame(performance_data)
    
    if df.empty:
        print("No performance data to visualize.")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Performance Analysis: Original vs Pruned Models', fontsize=16, fontweight='bold')
    
    models = df['model'].unique()
    
    # Plot 1: Accuracy vs Inference Time
    for model in models:
        model_data = df[df['model'] == model]
        
        original_data = model_data[model_data['variant'] == 'Original']
        pruned_data = model_data[model_data['variant'] != 'Original']
        
        if not original_data.empty:
            axes[0, 0].scatter(original_data['inference_time_ms'], original_data['accuracy'], 
                             s=150, alpha=0.7, marker='o', label=f'{model} Original')
        
        if not pruned_data.empty:
            axes[0, 0].scatter(pruned_data['inference_time_ms'], pruned_data['accuracy'], 
                             s=150, alpha=0.7, marker='s', label=f'{model} Pruned')
    
    axes[0, 0].set_title('Accuracy vs Inference Time', fontweight='bold')
    axes[0, 0].set_xlabel('Inference Time (ms)')
    axes[0, 0].set_ylabel('Accuracy (%)')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: Memory Usage Comparison
    if len(models) > 0:
        x_pos = np.arange(len(models))
        width = 0.35
        
        original_memory = []
        pruned_memory = []
        
        for model in models:
            model_data = df[df['model'] == model]
            
            orig_data = model_data[model_data['variant'] == 'Original']
            prun_data = model_data[model_data['variant'] != 'Original']
            
            original_memory.append(orig_data['total_memory_mb'].iloc[0] if not orig_data.empty else 0)
            pruned_memory.append(prun_data['total_memory_mb'].iloc[0] if not prun_data.empty else 0)
        
        axes[0, 1].bar(x_pos - width/2, original_memory, width, label='Original', alpha=0.8)
        axes[0, 1].bar(x_pos + width/2, pruned_memory, width, label='Pruned', alpha=0.8)
        
        axes[0, 1].set_title('Memory Usage Comparison', fontweight='bold')
        axes[0, 1].set_xlabel('Models')
        axes[0, 1].set_ylabel('Memory Usage (MB)')
        axes[0, 1].set_xticks(x_pos)
        axes[0, 1].set_xticklabels(models)
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Speedup Analysis
    pruned_only = df[df['variant'] != 'Original']
    if not pruned_only.empty:
        model_names = pruned_only['model']
        speedups = pruned_only['speedup']
        
        bars = axes[1, 0].bar(range(len(model_names)), speedups, alpha=0.8)
        axes[1, 0].axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='No speedup')
        axes[1, 0].set_title('Inference Speedup from Pruning', fontweight='bold')
        axes[1, 0].set_xlabel('Models')
        axes[1, 0].set_ylabel('Speedup Factor')
        axes[1, 0].set_xticks(range(len(model_names)))
        axes[1, 0].set_xticklabels(model_names)
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        # Add value labels
        for i, (bar, speedup) in enumerate(zip(bars, speedups)):
            height = bar.get_height()
            axes[1, 0].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                           f'{speedup:.2f}x', ha='center', va='bottom')
    
    # Plot 4: Efficiency Score (Accuracy per MB)
    efficiency_data = []
    labels = []
    
    for _, row in df.iterrows():
        if row['total_memory_mb'] > 0:
            efficiency = row['accuracy'] / row['total_memory_mb']
            efficiency_data.append(efficiency)
            labels.append(f"{row['model']}\n{row['variant']}")
    
    if efficiency_data:
        bars = axes[1, 1].bar(range(len(efficiency_data)), efficiency_data, alpha=0.8)
        axes[1, 1].set_title('Model Efficiency\n(Accuracy per MB)', fontweight='bold')
        axes[1, 1].set_xlabel('Model Variants')
        axes[1, 1].set_ylabel('Efficiency (Acc/MB)')
        axes[1, 1].set_xticks(range(len(labels)))
        axes[1, 1].set_xticklabels(labels, rotation=45, ha='right')
        axes[1, 1].grid(True, alpha=0.3)
        
        # Add value labels
        for i, (bar, eff) in enumerate(zip(bars, efficiency_data)):
            height = bar.get_height()
            axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                           f'{eff:.2f}', ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    plt.show()

# Run comprehensive performance analysis
print("INFERENCE SPEED AND MEMORY ANALYSIS")
print("=" * 40)

if models and pruned_models_50:
    # Run performance analysis
    performance_results = comprehensive_performance_analysis(models, pruned_models_50)
    
    # Visualize results
    visualize_performance_analysis(performance_results)
    
    # Create summary table
    perf_df = pd.DataFrame(performance_results)
    
    print("\nPerformance Analysis Summary:")
    print("=" * 40)
    
    summary_cols = ['model', 'variant', 'accuracy', 'inference_time_ms', 
                   'total_memory_mb', 'speedup', 'memory_reduction']
    
    display_df = perf_df[summary_cols].copy()
    display_df['accuracy'] = display_df['accuracy'].apply(lambda x: f"{x:.2f}%")
    display_df['inference_time_ms'] = display_df['inference_time_ms'].apply(lambda x: f"{x:.2f}ms")
    display_df['total_memory_mb'] = display_df['total_memory_mb'].apply(lambda x: f"{x:.1f}MB")
    display_df['speedup'] = display_df['speedup'].apply(lambda x: f"{x:.2f}x")
    display_df['memory_reduction'] = display_df['memory_reduction'].apply(lambda x: f"{x:.1f}%")
    
    print(display_df.to_string(index=False))
    
    # Performance improvements summary
    print("\nPerformance Improvements from Pruning:")
    print("-" * 45)
    
    for model in perf_df['model'].unique():
        model_data = perf_df[perf_df['model'] == model]
        pruned_data = model_data[model_data['variant'] != 'Original']
        
        if not pruned_data.empty:
            pruned_row = pruned_data.iloc[0]
            print(f"\n{model}:")
            print(f"  Inference speedup: {pruned_row['speedup']:.2f}x")
            print(f"  Memory reduction: {pruned_row['memory_reduction']:.1f}%")
            print(f"  Parameter reduction: {(1 - pruned_row['parameters'] / perf_df[perf_df['model'] == model]['parameters'].iloc[0]) * 100:.1f}%")
            
            if pruned_row['speedup'] > 1.1:
                print(f"  ✅ Significant speedup achieved")
            elif pruned_row['speedup'] > 1.0:
                print(f"  ⚠️  Modest speedup achieved")
            else:
                print(f"  ❌ No speedup (may be due to overhead)")

else:
    print("No models available for performance analysis.")

print("\nPerformance analysis complete!")

## 10. Summary and Conclusions

Summarize pruning analysis results and provide practical recommendations.

In [ ]:
print("\n" + "="*80)
print("MODEL PRUNING ANALYSIS - SUMMARY AND CONCLUSIONS")
print("="*80)

# Comprehensive summary based on all analyses
print("\n🔍 KEY FINDINGS:")
print("-" * 20)

# 1. Pruning Effectiveness
print("\n1. Pruning Effectiveness:")
if 'pruning_df' in locals() and not pruning_df.empty:
    # Find best performing pruning configurations
    best_results = []
    
    for model in pruning_df['model_name'].unique():
        model_data = pruning_df[pruning_df['model_name'] == model]
        
        # Find configuration with best accuracy retention at high compression
        high_compression = model_data[model_data['pruning_ratio'] >= 0.5]
        if not high_compression.empty:
            best_config = high_compression.loc[high_compression['pruned_accuracy'].idxmax()]
            best_results.append(best_config)
    
    if best_results:
        avg_compression = np.mean([r['compression_ratio'] for r in best_results])
        avg_acc_drop = np.mean([r['accuracy_drop'] for r in best_results])
        
        print(f"   • Average achievable compression: {avg_compression:.1f}x")
        print(f"   • Average accuracy drop: {avg_acc_drop:.2f}%")
        
        if avg_acc_drop < 5:
            print(f"   ✅ Excellent pruning results with minimal accuracy loss")
        elif avg_acc_drop < 10:
            print(f"   ✅ Good pruning results with acceptable accuracy loss")
        else:
            print(f"   ⚠️  Moderate pruning results with noticeable accuracy loss")
        
        # Model comparison
        print(f"\n   Model-specific results:")
        for result in best_results:
            print(f"     {result['model_name']}: {result['compression_ratio']:.1f}x compression, "
                  f"{result['accuracy_drop']:.2f}% accuracy drop")

# 2. Layer-wise Impact
print("\n2. Layer-wise Pruning Impact:")
if 'layer_stats_dict' in locals():
    for model_name, layer_stats in layer_stats_dict.items():
        conv_sparsities = [stat['sparsity'] for stat in layer_stats if stat['type'] == 'Conv2d']
        linear_sparsities = [stat['sparsity'] for stat in layer_stats if stat['type'] == 'Linear']
        
        if conv_sparsities and linear_sparsities:
            avg_conv_sparsity = np.mean(conv_sparsities) * 100
            avg_linear_sparsity = np.mean(linear_sparsities) * 100
            
            print(f"   • {model_name}:")
            print(f"     Conv layers: {avg_conv_sparsity:.1f}% average sparsity")
            print(f"     Linear layers: {avg_linear_sparsity:.1f}% average sparsity")
            
            if abs(avg_conv_sparsity - avg_linear_sparsity) < 10:
                print(f"     ✓ Uniform pruning across layer types")
            else:
                print(f"     ⚠️  Non-uniform pruning (may indicate layer sensitivity)")

# 3. Robustness Impact
print("\n3. Adversarial Robustness Impact:")
if 'robustness_comparison' in locals():
    robustness_changes = [r['change'] for r in robustness_comparison]
    
    if robustness_changes:
        avg_change = np.mean(robustness_changes)
        print(f"   • Average robustness change: {avg_change:+.2f}%")
        
        if avg_change > 1:
            print(f"   ✅ Pruning improved adversarial robustness")
        elif avg_change > -2:
            print(f"   ✓ Minimal impact on adversarial robustness")
        else:
            print(f"   ⚠️  Pruning reduced adversarial robustness")
        
        # Per-attack analysis
        attack_summary = {}
        for r in robustness_comparison:
            if r['attack'] not in attack_summary:
                attack_summary[r['attack']] = []
            attack_summary[r['attack']].append(r['change'])
        
        print(f"   Attack-specific changes:")
        for attack, changes in attack_summary.items():
            avg_attack_change = np.mean(changes)
            print(f"     {attack}: {avg_attack_change:+.2f}%")
else:
    print(f"   • No robustness analysis data available")

# 4. Performance Improvements
print("\n4. Performance Improvements:")
if 'performance_results' in locals():
    pruned_results = [r for r in performance_results if r['variant'] != 'Original']
    
    if pruned_results:
        avg_speedup = np.mean([r['speedup'] for r in pruned_results])
        avg_memory_reduction = np.mean([r['memory_reduction'] for r in pruned_results])
        
        print(f"   • Average inference speedup: {avg_speedup:.2f}x")
        print(f"   • Average memory reduction: {avg_memory_reduction:.1f}%")
        
        if avg_speedup > 1.5:
            print(f"   ✅ Significant performance improvements")
        elif avg_speedup > 1.1:
            print(f"   ✓ Moderate performance improvements")
        else:
            print(f"   ⚠️  Limited performance improvements (overhead may dominate)")
else:
    print(f"   • No performance analysis data available")

# 5. Advanced Techniques
print("\n5. Advanced Pruning Techniques:")
if 'gradual_results' in locals():
    final_gradual = gradual_results[-1]
    initial_acc = gradual_results[0]['accuracy']
    
    gradual_acc_drop = initial_acc - final_gradual['accuracy']
    
    print(f"   • Gradual pruning results:")
    print(f"     Final compression: {final_gradual['compression_ratio']:.1f}x")
    print(f"     Accuracy drop: {gradual_acc_drop:.2f}%")
    
    # Compare with one-shot if available
    if 'oneshot_acc' in locals():
        oneshot_drop = initial_acc - oneshot_acc
        improvement = oneshot_drop - gradual_acc_drop
        
        print(f"     vs One-shot pruning: {improvement:+.2f}% better accuracy retention")
        
        if improvement > 1:
            print(f"   ✅ Gradual pruning shows clear advantages")
        else:
            print(f"   ⚠️  Gradual vs one-shot pruning shows similar results")

# Practical Recommendations
print("\n📋 PRACTICAL RECOMMENDATIONS:")
print("-" * 35)

print("\n1. Optimal Pruning Strategy:")
if 'pruning_df' in locals() and not pruning_df.empty:
    # Find sweet spot for each model
    for model in pruning_df['model_name'].unique():
        model_data = pruning_df[pruning_df['model_name'] == model]
        
        # Find best balance (compression vs accuracy)
        model_data = model_data.copy()
        model_data['efficiency_score'] = model_data['compression_ratio'] / (1 + model_data['accuracy_drop'])
        best_config = model_data.loc[model_data['efficiency_score'].idxmax()]
        
        print(f"   • {model}: {best_config['pruning_ratio']:.1f} pruning ratio")
        print(f"     Achieves {best_config['compression_ratio']:.1f}x compression with {best_config['accuracy_drop']:.2f}% accuracy drop")

print("\n2. Implementation Guidelines:")
print("   • Start with magnitude-based unstructured pruning")
print("   • Use gradual pruning for better accuracy retention")
print("   • Target 50-70% sparsity for good compression without major accuracy loss")
print("   • Monitor both clean accuracy and adversarial robustness")
print("   • Consider layer sensitivity when applying structured pruning")

print("\n3. Hardware Considerations:")
print("   • Unstructured pruning may not always improve inference speed")
print("   • Structured pruning typically provides better speedups")
print("   • Memory reduction is often more consistent than speed improvements")
print("   • Consider target deployment hardware when choosing pruning strategy")

print("\n4. Model-Specific Insights:")
if 'pruning_df' in locals() and not pruning_df.empty:
    for model in pruning_df['model_name'].unique():
        model_data = pruning_df[pruning_df['model_name'] == model]
        max_compression = model_data['compression_ratio'].max()
        min_acc_drop = model_data[model_data['compression_ratio'] > 2]['accuracy_drop'].min()
        
        print(f"   • {model}:")
        print(f"     Max compression: {max_compression:.1f}x")
        print(f"     Min accuracy drop at >2x compression: {min_acc_drop:.2f}%")
        
        if min_acc_drop < 5:
            print(f"     ✓ Highly compressible model")
        elif min_acc_drop < 10:
            print(f"     ✓ Moderately compressible model")
        else:
            print(f"     ⚠️  Less compressible - careful tuning needed")

# Future Work and Advanced Techniques
print("\n🔬 FUTURE WORK AND ADVANCED TECHNIQUES:")
print("-" * 45)

print("\n1. Advanced Pruning Methods:")
print("   • Lottery ticket hypothesis - find winning subnetworks")
print("   • SNIP (Single-shot Network Pruning at Initialization)")
print("   • Magnitude-based structured pruning with automatic search")
print("   • Neural architecture search for optimal sparse architectures")

print("\n2. Hardware-Aware Pruning:")
print("   • Target specific hardware accelerators (TPUs, mobile chips)")
print("   • Consider memory bandwidth and cache optimization")
print("   • Balance between model size and computational complexity")

print("\n3. Joint Optimization:")
print("   • Combine pruning with quantization for maximum compression")
print("   • Knowledge distillation from unpruned to pruned models")
print("   • Adversarial training on pruned models for robust compression")

print("\n4. Domain-Specific Considerations:")
print("   • Sports classification may benefit from preserving motion-related features")
print("   • Consider pruning impact on important visual features")
print("   • Evaluate pruning on different sports categories")

# Implementation Priority
print("\n📊 IMPLEMENTATION PRIORITY:")
print("-" * 30)

if 'pruning_df' in locals() and not pruning_df.empty:
    avg_accuracy_drop = pruning_df[pruning_df['pruning_ratio'] == 0.5]['accuracy_drop'].mean()
    
    print(f"\nCurrent pruning performance at 50% sparsity: {avg_accuracy_drop:.2f}% average accuracy drop")
    
    if avg_accuracy_drop < 3:
        print("\n🚀 HIGH PRIORITY - Excellent pruning candidates:")
        print("   1. Deploy pruned models immediately")
        print("   2. Explore higher compression ratios")
        print("   3. Implement structured pruning for hardware optimization")
    elif avg_accuracy_drop < 7:
        print("\n✅ MEDIUM PRIORITY - Good pruning potential:")
        print("   1. Fine-tune pruning ratios per layer")
        print("   2. Implement gradual pruning")
        print("   3. Consider accuracy-robustness trade-offs")
    else:
        print("\n⚠️  LOW PRIORITY - Challenging pruning case:")
        print("   1. Focus on improving base model first")
        print("   2. Explore alternative compression methods")
        print("   3. Consider knowledge distillation")

print("\n" + "="*80)
print("🎯 Model pruning analysis complete!")
print("📊 Use these insights to optimize model deployment efficiency.")
print("⚡ Remember: The best pruning strategy depends on your specific deployment constraints.")
print("="*80)